<a href="https://colab.research.google.com/github/viniciusdutra314/CardMaker/blob/main/CardMaker_refactoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installing and importing modules

In [ ]:
!pip install colorama
!pip install genanki
!pip install googletrans==3.1.0a0
!pip install gtts
!pip install openpyxl
from colorama import Fore,Back,Style,init #use colors
import genanki #integration with anki
from googletrans import Translator #translate the phrases
from openpyxl import load_workbook
from gtts import gTTS #generate audios
import os #get the file directories
import time #count the amount of time used in creating cards
from zipfile import ZipFile #for some reason google colab don't download folders, so i'll convert the audio folder into winrar

## **How to use it**
1. Import your excel table to the file menu
2. Run the "Installing and import modules" (may take a minute)
3. Click in "Run CardMaker"
4. Use the code as explained on my [Github](https://github.com/viniciusdutra314/CardMaker)
5. The code will generate an "audios_table.zip", you need to open it and past these audios file into the collection.media (example of directory C:\Users\dutra\AppData\Roaming\Anki2\Dutra\collection.media)
6. You also need to open the "table.apkg" file which is an anki deck

In [57]:
translator = Translator() #start translator
init(strip=False) #colors in the terminal
t_o=time.time() # counting time

def translate(message,source="en"):
  global chosen_language
  return translator.translate(message, src=source,dest=chosen_language).text

chosen_language=input("Choose the language that your cards and the interface will be translated (ex:pt,fr,en): ") 

tables_in_the_directory=[j for j in os.listdir() if j[-5::]==".xlsx" and j[0:2]!="~$"]

if len(tables_in_the_directory)>1:  #multiple tables to select
  print(tables_in_the_directory)
  while True:
    table_name=input(translate("Name of the excel table"))+".xlsx"
    if table_name in tables_in_the_directory: break
    else:
      wrong_name=translate("Wrong name!")
      print(Back.RED + wrong_name,Style.RESET_ALL,end="")
else: 
  table_name=tables_in_the_directory[0]
  print(translate("Using the table")+" " +Back.GREEN +table_name+Style.RESET_ALL)
table ,deck_name= load_workbook(table_name).active, table_name[:-5:]
print("p=pronunciation,s=speaking,w=writing,v=vocabulary")
cardtype=input(translate("What is the type of your card?")).lower()

phrases = [cell.value for cell in table['A'] if cell.value!=None][1::]
if cardtype=="v": 
  words=[cell.value for cell in table['B'] if cell.value!=None][1::]

langs=["lang","l","languages","language"]

if table['B1'].value.lower() or table['C1'].value.lower() in langs:
    column="B" if table['B1'].value.lower() in langs else "C"
    languages=[cell.value for cell in table[column] if cell.value!=None][1::]
    if len(languages)==1:
      languages=[unique_lang:=languages[0] for _ in range(len(phrases))]

def wordinphrase(word,phrase):
    word,phrase=str(word).lower(),str(phrase).lower()
    remove=[".",",","?","!",":",";","-","=","_"]
    for k in remove:
        word=word.replace(k,"")
        phrase=phrase.replace(k," ")
    if len(word.split())==1: #single word
      for j in phrase.split():
          if str(j)==str(word):
              return True
    else: #composite word
      if word in phrase: return True
    return False

if cardtype=='v':
    if input(translate("Do you want to check if the word is in the phrase?")+" y=yes, n=no:")[0].lower() =="y":
        for j in range(len(phrases)):
            if not wordinphrase(words[j],phrases[j]):
              while True:
                print(Back.RED + translate("Error: Word not in the phrase"),end="")
                print("\n"+Style.RESET_ALL+phrases[j]+"\n"+words[j])
                words[j]=input(translate("Correct word:"))
                if wordinphrase(words[j],phrases[j]):
                  break


print(translate("Translating...(1/2)"))

translated_phrases=[translate(phrases[i],source=languages[i]) for i in range(len(phrases))]
if cardtype=="v": translated_words=[translate(words[i],source=languages[i]) for i in range(len(phrases))]

#checking


print(translate("Audios...(2/2)"))

if not os.path.exists('tempaudios'):
  os.mkdir('tempaudios')
  audio_path="//content//tempaudios"

for j in range(len(phrases)):
  try:
    if cardtype=="v":
      audio = gTTS(words[j]+"."+phrases[j], lang=languages[j])
      audio.save(audio_path + "//" + deck_name + "phrase" + str(j) + '.mp3')
      audio= gTTS(words[j], lang=languages[j])
      audio.save(audio_path + "//" + deck_name + "word" + str(j) + '.mp3')
    if cardtype in ["s","w"]:
      audio = gTTS(phrases[j], lang=languages[j])
      audio.save(audio_path + "//" + deck_name + "phrase" + str(j) + '.mp3')
    if cardtype=="p":
      audio = gTTS(phrases[j], lang=languages[j])
      audio.save(audio_path + "//" + deck_name + "pronunciation" + str(j) + '.mp3')
  except:pass

from random import randint
id_deck =randint(1_000_000_000, 10_000_000_000)
fields=[
            {'name': 'Question'},
            {'name': 'Answer'},
            {'name': 'MyMedia'},
        ]

deck = genanki.Deck(
    id_deck,
    deck_name)
if cardtype in ["v","s"]:
    my_model = genanki.Model(
        id_deck,
        'CardMaker Q&A',
        fields=fields,
        templates=[
            {
                'name': 'Card 1',
                'qfmt': '{{Question}}<br>{{MyMedia}}',
                'afmt': '{{FrontSide}}<hr id="answer">{{Answer}}',
            },
        ])
if cardtype=="w":
    my_model = genanki.Model(
        id_deck,
        'CardMaker Type in the Answer',
        fields=fields,
        templates=[
            {
                'name': 'Card 1',
                'qfmt': '{{Question}}<br> {{type:Answer}}',
                'afmt': '{{FrontSide}}{{MyMedia}}',
            },
        ])
if cardtype=="p":
    my_model = genanki.Model(
        id_deck,
        'CardMaker Pronunciation',
        fields=fields,
        templates=[
            {
                'name': 'Card 1',
                'qfmt': '{{Question}}<br>{{MyMedia}}',
                'afmt': '{{FrontSide}}<hr id="answer">{{Answer}}',
            },
        ])
#Color of the cards

print(Fore.BLUE + "Blue",Fore.RED + "Red",Fore.YELLOW + "Yellow",Fore.MAGENTA + "Purple",Fore.GREEN + "Green",Fore.LIGHTMAGENTA_EX + "Pink"+Style.RESET_ALL)
color=input(translate("Select a color: "))[0:2].lower()
colors_rgb = {'gr': '<span style="color: rgb(81, 255, 37);">', 're': '<span style="color: rgb(228, 14, 14);">',
              'bl': '<span style="color: rgb(18, 166, 252);">',
              'ye': '<span style="color: rgb(249, 255, 54);">',
              'pu': '<span style="color: rgb(198, 38, 255);">',
              'pi': '<span style="color: rgb(255, 14, 192);">', }

color=colors_rgb[color]

def insert_sound(file_name): return "[" + "sound:" + file_name + ".mp3" + "]"
def bold(text): return '<u><b><i>' + text +'</i></b></u></span>'
def create_note(text):
  global i,my_model,languages
  return genanki.Note(model=my_model,fields=text,tags=[str(languages[int(i)]), "cardmaker"])

for i in range(len(phrases)):
        if cardtype=="s":
            note =create_note([color + bold(languages[i]) +translated_phrases[i],insert_sound(deck_name + "phrase" + str(i)) + phrases[i],"" ])
        if cardtype=="v":
            note =create_note(["",insert_sound(deck_name + "word" + str(i)) + color + bold(words[i])+  " == " + translated_words[i],
                               insert_sound(deck_name + "phrase" + str(i))  + color+ bold(words[i]) + ". " +phrases[i]])
        if cardtype=="w":
            note = create_note([color + bold(languages[i])+translated_phrases[i], phrases[i],insert_sound(deck_name+"phrase"+str(i))])
        if cardtype=="p":
            note = create_note([color + bold(phrases[i]),insert_sound(deck_name + "pronunciation" + str(i))," ", ""])
        deck.add_note(note)

genanki.Package(deck).write_to_file(deck_name +'.apkg')

readme=open("readme.txt","w")
readme.write(translate("copy this audios into your collection.media anki folder"))
readme.close()

with ZipFile(f"audios_{deck_name}.zip","w") as zip:
  zip.write("readme.txt")
  for j in os.listdir("tempaudios/"):
    zip.write("tempaudios"+"//"+j)
    os.remove("//content//tempaudios"+"//"+j)
os.remove("readme.txt")
os.rmdir('tempaudios')

deltat=time.time() -t_o
print(f"Congratulations, {len(phrases)} flashcards in {round(deltat/60,1)} minutes! {round(60*len(phrases)/deltat,1)} flashcards per minute")


KeyboardInterrupt: ignored